In [1]:
import pickle as pk
import pandas as pd
import numpy as np
from m3gnet.models import M3GNet, Potential
from m3gnet.trainers import PotentialTrainer
import tensorflow as tf

print('loading the MPF dataset 2021')
with open('D:\Projects\projects_python\m3gnet_tests\\block_0.p', 'rb') as f:
    data = pk.load(f)

with open('D:\Projects\projects_python\m3gnet_tests\\block_1.p', 'rb') as f:
    data2 = pk.load(f)
print('MPF dataset 2021 loaded')
data.update(data2)
df = pd.DataFrame.from_dict(data)

loading the MPF dataset 2021
MPF dataset 2021 loaded


In [1]:
pip install pymatgen

  Using cached pymatgen-2022.0.17.tar.gz (40.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-1.21.6-cp37-cp37m-win_amd64.whl.metadata (2.2 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached ruamel.yaml-0.18.6-py3-none-any.whl.metadata (23 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached palettable-3.3.3-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached uncertainties-3.1.7-py2.py3-none-any.whl.metadata (14 kB)
     -------------------------------------- 138.5/138.5 kB 1.6 MB/s eta 0:00:00
  Using cached pypar

  error: subprocess-exited-with-error
  
  × Building wheel for pymatgen (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [1298 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-37
      creating build\lib.win-amd64-cpython-37\pymatgen
      creating build\lib.win-amd64-cpython-37\pymatgen\alchemy
      copying pymatgen\alchemy\filters.py -> build\lib.win-amd64-cpython-37\pymatgen\alchemy
      copying pymatgen\alchemy\materials.py -> build\lib.win-amd64-cpython-37\pymatgen\alchemy
      copying pymatgen\alchemy\transmuters.py -> build\lib.win-amd64-cpython-37\pymatgen\alchemy
      copying pymatgen\alchemy\__init__.py -> build\lib.win-amd64-cpython-37\pymatgen\alchemy
      creating build\lib.win-amd64-cpython-37\pymatgen\analysis
      copying pymatgen\analysis\adsorption.py -> build\lib.win-amd64-cpython-37\pymatgen\analysis
      copying pymatgen\analysis\bond_dissoc

In [2]:
dataset_train = []
for idx, item in df.items():
    for iid in range(len(item['energy'])):
        dataset_train.append({"atoms":item['structure'][iid], "energy":item['energy'][iid] / len(item['force'][iid]), "force": np.array(item['force'][iid]), "stress": np.array(item['stress'][iid])})

print('using %d samples to train, '%(len(dataset_train)))

using 187687 samples to train, 


In [3]:
dataset_train[0]["atoms"]

Structure Summary
Lattice
    abc : 9.00667017659015 8.486714308993259 12.38991512058143
 angles : 94.92204563912044 90.03854066865074 89.99436712154997
 volume : 943.555805370328
      A : -7.8013173 -4.50105389 0.00836265
      B : -2.60378959 4.49880692 -6.70860163
      C : 5.2036832 -9.01493757 -6.72031089
PeriodicSite: Li (-7.1228, -5.0483, -4.9850) [0.9648, 0.4439, 0.2998]
PeriodicSite: Li (-6.4482, -2.1787, -6.7526) [0.7405, 0.7572, 0.2498]
PeriodicSite: Li (-4.5296, -3.9453, -8.4078) [0.6540, 0.7619, 0.4913]
PeriodicSite: Li (-3.9974, -6.7751, -10.1176) [0.7599, 0.7576, 0.7502]
PeriodicSite: Li (-3.8579, -1.4494, -5.0364) [0.4511, 0.5439, 0.2070]
PeriodicSite: Li (-3.9884, -10.4772, -5.0036) [0.9650, 0.0437, 0.7022]
PeriodicSite: Li (-3.4076, -9.3345, -11.7662) [0.8453, 0.7604, 0.9929]
PeriodicSite: Li (-3.2379, -4.0407, -1.6500) [0.5356, 0.0437, 0.2026]
PeriodicSite: Li (-1.8189, -4.8275, -11.9674) [0.4422, 0.9795, 0.8035]
PeriodicSite: Li (3.2946, -8.7013, -8.2406) [0.1660, 

In [4]:
m3gnet = M3GNet(is_intensive=False)
potential = Potential(model=m3gnet)

trainer = PotentialTrainer(
    potential=potential, optimizer=tf.keras.optimizers.Adam(1e-3)
)

In [5]:
trainer.train(
    [dataset_train[0]['atoms']],
    [dataset_train[0]['energy']],
    [dataset_train[0]['force']],
    validation_graphs_or_structures=[dataset_train[0]['atoms']],
    val_energies=[dataset_train[0]['energy']],
    val_forces=[dataset_train[0]['force']],
    epochs=2,
    fit_per_element_offset=True,
    save_checkpoint=False,
)

Epoch 1/2
1/1 [==============================] - 23s 23s/step - loss: 0.0022 - MAE(E): 0.0020 - MAE(F): 0.0361 - MAE(S): 0.0000e+00 - val_MAE: 0.0396 - val_MAE(E): 0.0035 - val_MAE(F): 0.0361 - val_MAE(S): 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 426ms/step - loss: 0.0022 - MAE(E): 0.0035 - MAE(F): 0.0361 - MAE(S): 0.0000e+00 - val_MAE: 0.0376 - val_MAE(E): 0.0015 - val_MAE(F): 0.0361 - val_MAE(S): 0.0000e+00


In [1]:
import warnings

from pymatgen.core import Lattice, Structure
from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")

# Init a Mo structure with stretched lattice (DFT lattice constant ~ 3.168)
mo = Structure(Lattice.cubic(3.3), ["Mo", "Mo"], [[0., 0., 0.], [0.5, 0.5, 0.5]])

relaxer = Relaxer()  # This loads the default pre-trained model

relax_results = relaxer.relax(mo, verbose=True)

final_structure = relax_results['final_structure']
final_energy_per_atom = float(relax_results['trajectory'].energies[-1] / len(mo))

print(f"Relaxed lattice parameter is {final_structure.lattice.abc[0]:.3f} Å")
print(f"Final energy is {final_energy_per_atom:.3f} eV/atom")

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 10:17:51      -21.330704*      10.2491
FIRE:    1 10:17:54      -20.743158*      20.4843
FIRE:    2 10:17:54      -21.694870*       2.4213
FIRE:    3 10:17:54      -21.208538*      11.5265
FIRE:    4 10:17:54      -21.353798*       9.9677
FIRE:    5 10:17:54      -21.562597*       6.5603
FIRE:    6 10:17:54      -21.702246*       2.0209
FIRE:    7 10:17:54      -21.696190*       2.3511
FIRE:    8 10:17:54      -21.697779*       2.2639
FIRE:    9 10:17:54      -21.700720*       2.0934
FIRE:   10 10:17:54      -21.704586*       1.8471
FIRE:   11 10:17:54      -21.708820*       1.5354
FIRE:   12 10:17:54      -21.712812*       1.1708
FIRE:   13 10:17:54      -21.716003*       0.7668
FIRE:   14 10:17:54      -21.717947*       0.3373
FIRE:   15 10:17:54      -21.718319*       0.1499
FIRE:   16 10:17:54      -21.718323*       0.1483
FIRE:   17 10:17:54      -21.718328*       0.1450
FI

In [4]:
pip install m3gnet

  Obtaining dependency information for m3gnet from https://files.pythonhosted.org/packages/b0/99/4fd58fcd278113191e43c57e208fd31c047920610df59d980cc863028da9/m3gnet-0.2.4-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for ase from https://files.pythonhosted.org/packages/38/b0/3c0a7afaf66274588216c251376ac2bea0269eb7a5e1da77521811060553/ase-3.22.1-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pickle as pk
import pandas as pd
import numpy as np
from m3gnet.models import M3GNet, Potential
from m3gnet.trainers import PotentialTrainer
import tensorflow as tf
from pymatgen.core.lattice import Lattice
from pymatgen.core.sites import PeriodicSite
from pymatgen.core.structure import Structure, Molecule

In [4]:
data = pd.read_csv('data.csv')

In [3]:
data

,xyz,U_0,id
0,35\n\nH 87.320200 76.885300 54.789800\nH 92.14...,-657.273213,0
1,12\n\nH 7.371870 29.624600 44.150900\nH 6.3842...,-231.611281,1
2,16\n\nC 58.005900 19.795900 43.264800\nC 61.83...,-308.894707,2
3,8\n\nC 19.086700 70.324800 42.711300\nC 19.325...,-154.457101,3
4,16\n\nH 46.064800 44.166400 40.079200\nH 47.87...,-308.919864,4
...,...,...,...
78402,13\n\nH 54.696800 10.533200 37.066600\nC 54.86...,-269.731878,78402
78403,22\n\nC 41.839600 82.360100 17.805200\nH 42.61...,-462.366388,78403
78404,10\n\nH 38.478500 85.525600 6.027890\nH 36.877...,-155.704251,78404
78405,19\n\nC 42.663200 58.273700 39.489300\nH 42.46...,-423.132241,78405


In [5]:
data = pd.read_csv('data.csv')
molecule = Molecule.from_str(data.iloc[0]['xyz'], 'xyz')
# убираем относительные координаты
coords = molecule.cart_coords - molecule.cart_coords[0]
structure = Structure(Lattice.cubic(1), molecule.species, coords)

In [13]:
lattice = Lattice.from_parameters(a=1, b=1, c=1, alpha=90, beta=90, gamma=90)

# Получите координаты атомов из объекта molecule
coords = molecule.cart_coords - molecule.cart_coords[0]

# Создайте объект Structure, передавая ячейку и координаты атомов
structure = Structure(Lattice.cubic(1), molecule.species, coords)

In [7]:
m3gnet = M3GNet(is_intensive=False)
potential = Potential(model=m3gnet)

trainer = PotentialTrainer(
    potential=potential, optimizer=tf.keras.optimizers.Adam(1e-3)
)

In [6]:
data.iloc[0]['U_0']

-657.2732132581

In [9]:
trainer.train(
    [structure],
    [data.iloc[0]['U_0']],
    [1],
    validation_graphs_or_structures=[structure],
    val_energies=[data.iloc[0]['U_0']],
    val_forces=[1],
    epochs=2,
    fit_per_element_offset=False,
    save_checkpoint=False,
)

OverflowError: Python int too large to convert to C long

In [2]:
from pymatgen.core.structure import Structure, Molecule
from pymatgen.core.lattice import Lattice
from pymatgen.core.sites import PeriodicSite
from io import StringIO

def xyz_str_to_structure(xyz_str):
    lines = xyz_str.strip().split('\n')

    # Извлечение числа атомов
    num_atoms = int(lines[0])

    # Извлечение строк с данными атомов
    atom_lines = lines[2:num_atoms+2]

    # Разделение данных атомов на имена и координаты
    atoms = []
    coords = []
    for line in atom_lines:
        parts = line.strip().split()
        atom = parts[0]
        x, y, z = map(float, parts[1:])
        atoms.append(atom)
        coords.append([x, y, z])

    # Создание объекта Structure с использованием полученных данных
    lattice = Lattice.from_parameters(1, 1, 1, 90, 90, 90)  # Здесь указывайте размеры сетки вместо 1, 1, 1
    periodic_sites = []
    for coord, atom in zip(coords, atoms):
        periodic_sites.append(PeriodicSite(atom, coord, lattice))

    structure = Structure(lattice, periodic_sites)

    return structure

structure = xyz_str_to_structure(data.iloc[0]['xyz'])

# Вывод структуры
print(structure)




NameError: name 'data' is not defined